In [1]:
import sys
sys.path.insert(0, '/home/steve/Dokumenty/Studia/project/ucsgnet-students-version')
sys.path.insert(0, '/home/steve/Dokumenty/Studia/project/CSGNet')

from ucsgnet.dataset import CADDataset, get_simple_2d_transforms
import matplotlib.pyplot as plt
from ucsgnet.ucsgnet.cad.net_cad import Net
import torch
from torch.utils.data import DataLoader
from tqdm import tqdm
import math
import argparse

In [2]:
params_path = '/home/steve/Dokumenty/Studia/project/ucsgnet-students-version/models/pretrained/models/cad_main/initial/params.json'
weights_path = '/home/steve/Dokumenty/Studia/project/ucsgnet-students-version/models/pretrained/models/cad_main/initial/ckpts/model.ckpt'
synthetic_dir = '/home/steve/Dokumenty/Studia/project/ucsgnet-students-version/data/synthetic'
cad_path = '/home/steve/Dokumenty/Studia/project/ucsgnet-students-version/data/cad/cad.h5'

In [3]:
dataset = CADDataset(cad_path, '', get_simple_2d_transforms())
loader = DataLoader(
            dataset=dataset,
            batch_size=1, # 16
            shuffle=False,
            drop_last=False,
            num_workers=0,
        )
len(dataset), len(loader)

(3000, 3000)

In [4]:
dataset[0][0].shape
# dataset[1][2]

torch.Size([1, 64, 64])

In [8]:
hparams_dict = {
    'use_planes': False,
    'shapes_per_type': 16,
    'num_dimensions': 2,
    'out_shapes_per_layer': 4,
    'weight_binarizing_threshold': 0.1,
    'num_csg_layers': 2,
}
hparams = argparse.Namespace()
for key, value in hparams_dict.items():
    setattr(hparams, key, value)
net = Net(hparams)
# hparams
net.net

Num of trainable params: 5709491
Num of not trainable params: 0


CSGNet(
  (encoder_): Extractor2D(
    (layers): Sequential(
      (0): Conv2DWithActivation(
        (conv): Conv2d(1, 32, kernel_size=(4, 4), stride=(2, 2), padding=(2, 2))
        (lrelu): LeakyReLU(negative_slope=0.01)
      )
      (1): Conv2DWithActivation(
        (conv): Conv2d(32, 64, kernel_size=(4, 4), stride=(2, 2), padding=(2, 2))
        (lrelu): LeakyReLU(negative_slope=0.01)
      )
      (2): Conv2DWithActivation(
        (conv): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(2, 2))
        (lrelu): LeakyReLU(negative_slope=0.01)
      )
      (3): Conv2DWithActivation(
        (conv): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(2, 2))
        (lrelu): LeakyReLU(negative_slope=0.01)
      )
      (4): Conv2DWithoutActivation(
        (conv): Conv2d(256, 256, kernel_size=(4, 4), stride=(2, 2))
      )
      (5): Flatten()
    )
  )
  (decoder_): Decoder(
    (layers): Sequential(
      (0): Linear(in_features=256, out_features=512, bias=Tru

In [9]:
# net = Net.load_from_checkpoint(weights_path)
# net.eval()
# net.freeze()
# net.net
# net = Net(
    
# )

In [10]:
prev = None
counter = {'t': 0, 'f': 0}
for batch in tqdm(loader):
    image, points, trues, _ = batch
#     if prev is not None:
#         if torch.equal(prev, points):
#             counter['t'] += 1
#         else:
#             counter['f'] += 1
#     prev = points
    hidden = net.net.encoder_(image)
    out = net.net.decoder_(hidden)
#     out = net(image, points)
    points = points.unsqueeze(
        dim=1
    )  # broadcasting for different of shapes

    base_shapes = net.net.evaluator_(
        out, points
    )  # -> batch, num_shapes, num_points
#     base_shapes_square = net.net.evaluator_.parts[1](
#         out, points
#     )

    base_shapes = base_shapes.permute(
        (0, 2, 1)
    )  # -> batch, num_points, num_shapes
    break
# counter

  0%|                                                                                                                                                                               | 0/3000 [00:00<?, ?it/s]


In [ ]:
# base_shapes_circle

In [11]:
base_shapes.shape

torch.Size([1, 4096, 48])

In [ ]:
# points.size(), points.min() + 0.0078

In [ ]:
# q_points = points.abs()
# q_points.max(torch.zeros_like(q_points)).norm(dim=-1)